In [14]:
import polars as pl
from sklearn import neighbors, preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, mean_squared_error, roc_auc_score
import math
from sklearn.ensemble import BaggingRegressor
from imblearn import under_sampling
from sklearnex import patch_sklearn

patch_sklearn() # this only works for intel devices

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [3]:
df = pl.read_csv("./data/EEG_Eye_State_Classification.csv")
df.head()

AF3,F7,F3,FC5,T7,P7,O1,O2,P8,T8,FC6,F4,F8,AF4,eyeDetection
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64
4329.23,4009.23,4289.23,4148.21,4350.26,4586.15,4096.92,4641.03,4222.05,4238.46,4211.28,4280.51,4635.9,4393.85,0
4324.62,4004.62,4293.85,4148.72,4342.05,4586.67,4097.44,4638.97,4210.77,4226.67,4207.69,4279.49,4632.82,4384.1,0
4327.69,4006.67,4295.38,4156.41,4336.92,4583.59,4096.92,4630.26,4207.69,4222.05,4206.67,4282.05,4628.72,4389.23,0
4328.72,4011.79,4296.41,4155.9,4343.59,4582.56,4097.44,4630.77,4217.44,4235.38,4210.77,4287.69,4632.31,4396.41,0
4326.15,4011.79,4292.31,4151.28,4347.69,4586.67,4095.9,4627.69,4210.77,4244.1,4212.82,4288.21,4632.82,4398.46,0


In [4]:
x_train, x_test, y_train, y_test = train_test_split(
    df.drop("eyeDetection").to_numpy(),
    df.select("eyeDetection").to_numpy(),
    test_size=0.2,
    random_state=42,
)
# tl = under_sampling.TomekLinks()
# x_train, y_train = tl.fit_resample(x_train, y_train)

In [9]:
params = {"n_neighbors": range(1, 50), "weights": ["uniform", "distance"]}
best_params = (
    GridSearchCV(neighbors.KNeighborsClassifier(), params)
    .fit(x_train, y_train)
    .best_params_
)

/Users/oli4/Documents/programming/python/ml-tutor/.venv/lib/python3.11/site-packages/onedal/datatypes/validation.py:170: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = _column_or_1d(y, warn=True)
/Users/oli4/Documents/programming/python/ml-tutor/.venv/lib/python3.11/site-packages/onedal/datatypes/validation.py:170: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = _column_or_1d(y, warn=True)
/Users/oli4/Documents/programming/python/ml-tutor/.venv/lib/python3.11/site-packages/onedal/datatypes/validation.py:170: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = _column_or_1d(y, warn=True)
/Users/oli4/Documents/programming/python/ml-tutor/.venv/lib/python3

In [12]:
model = BaggingRegressor(neighbors.KNeighborsClassifier(**best_params), n_estimators=10)
model.fit(x_train, y_train)

/Users/oli4/Documents/programming/python/ml-tutor/.venv/lib/python3.11/site-packages/onedal/datatypes/validation.py:170: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = _column_or_1d(y, warn=True)


{'n_neighbors': 1, 'weights': 'uniform'}

In [15]:
y_pred =  model.predict(x_test)
acc = accuracy_score(y_test, y_pred)
rmse_train = math.sqrt(mean_squared_error(model.predict(x_train), y_train))
rmse_test = math.sqrt(mean_squared_error(y_test, y_pred))

rmse_train, rmse_test, acc, roc_auc_score(y_test, y_pred)

(0.00913480119052337,
 0.15285446012893575,
 0.9766355140186916,
 0.9762397932261901)